In [1]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

Using TensorFlow backend.


11501568/11490434 [==============================] - 3s 0us/step


In [0]:
# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

In [0]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [0]:
from keras.callbacks import TensorBoard
from time import time
model = Sequential()
model.add(Dense(1000, input_dim=784, activation='sigmoid'))
model.add(Dense(10,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
tensorboard=TensorBoard(log_dir='logs/tensor9')
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2,callbacks=[tensorboard])

In [0]:
from google.datalab.ml import TensorBoard as tb
tb.start('logs/tensor9')

In [0]:
from keras import backend as K
K.clear_session()
from keras.callbacks import TensorBoard
from time import time
model = Sequential()
model.add(Dense(num_pixels, input_dim=784, kernel_initializer='normal',activation='sigmoid',name='input_layer1'))
model.add(Dense(1000,  activation='relu',name='hidden_layer1'))
model.add(Dense(10,  activation='softmax',name='output_layer1'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
tensorboard=TensorBoard(log_dir='logs/tensor10',histogram_freq=1,batch_size=10000)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=1024, verbose=2,callbacks=[tensorboard])

In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
data.test.cls = np.argmax(data.test.labels, axis=1)
data.train.cls = np.argmax(data.train.labels, axis=1)

In [0]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x2": np.array(data.train.images)},
    y=np.array(data.train.cls),
    num_epochs=None,
    batch_size=1024,
    shuffle=True)

In [0]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x2": np.array(data.test.images)},
    y=np.array(data.test.cls),
    num_epochs=1,
    shuffle=False)

In [0]:
feature_x = tf.feature_column.numeric_column("x2", shape=(784))

In [10]:
feature_columns = [feature_x]
feature_columns

[_NumericColumn(key='x2', shape=(784,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
def model_fn(features, labels, mode, params):
  x = features["x2"]
  net = tf.layers.dense(inputs=x, name='h1',units=512, activation=tf.nn.relu)
  net2 = tf.layers.dense(inputs=net, name='h2',units=256, activation=tf.nn.relu)
  net3 = tf.layers.dense(inputs=net2, name='h3',units=128, activation=tf.nn.relu)
  net4 = tf.layers.dense(inputs=net3, name='softmax',units=10,activation=tf.nn.softmax)
  
  y_pred_cls = tf.argmax(net4, axis=1)
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    spec = tf.estimator.EstimatorSpec(mode=mode,predictions=y_pred_cls)
    
  else:
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits = net4)
    loss = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=params["learning_rate"],l1_regularization_strength=0.001)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    accuracy = tf.metrics.accuracy(labels, y_pred_cls)
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('train_accuracy', accuracy[1])
    
    tf.summary.histogram("hidden1",net)
    tf.summary.histogram("hidden2",net2)
    tf.summary.histogram("hidden3",net3)
    
    spec = tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op,eval_metric_ops=metrics)
    
  return spec

In [0]:
params = {"learning_rate": 0.1}
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,model_dir='/content/datalab/docs/log11/')

In [0]:
model.train(input_fn=train_input_fn, steps=1000)

In [0]:
from google.datalab.ml import TensorBoard as tb
tb.start('/content/datalab/docs/log11/')

In [13]:
num_hidden_units = [512, 256, 128]
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                   hidden_units=num_hidden_units,
                                   activation_fn=tf.nn.relu,
                                   n_classes=num_classes,
                                  model_dir='/content/datalab/docs/log4')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe96aff8910>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/content/datalab/docs/log4', '_save_summary_steps': 100}


In [0]:
model.train(input_fn=train_input_fn, steps=2000)

In [0]:
from google.datalab.ml import TensorBoard as tb

In [0]:
tb.start('/content/datalab/docs/log4/')